## Project Overview

This project demonstrates sentiment analysis using the IMDB movie reviews dataset. It leverages the Hugging Face datasets library for data loading and preprocessing, and uses a pre-trained DistilBERT model from the Transformers library for fine-tuning.
## Workflow Summary

    Dataset: IMDB (from Hugging Face datasets)

    Model: Pre-trained DistilBERT (distilbert-base-uncased)

    Preprocessing: Tokenization using DistilBertTokenizer

    Fine-tuning: Hugging Face Trainer API with custom training arguments

    Evaluation: The model’s performance is evaluated on the test set primarily using accuracy as the key metric.

    Confusion Matrix: The confusion matrix is computed and printed during evaluation to help analyze prediction errors and class-wise performance.

## Inference

After fine-tuning, the model can be used to predict the sentiment of new text samples. The inference process involves tokenizing input sentences, passing them through the fine-tuned model, and interpreting the predicted class (positive or negative sentiment) along with confidence scores.

In [1]:
!pip install transformers
!pip install 'accelerate>=0.26.0'
!pip install -U datasets huggingface_hub
!pip install fsspec==2023.9.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

In [2]:
from datasets import load_dataset

dataset = load_dataset("imdb", download_mode="force_redownload")

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
#Step 2: Tokenize the Dataset
from transformers import DistilBertTokenizer
# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
print(tokenized_datasets["train"][0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [5]:
#Load pre trained model
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

2025-08-19 00:14:55.201097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755562495.538334      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755562495.633602      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
#Step 4: Define Training Arguments
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
output_dir="./results", eval_strategy="epoch",
    learning_rate=2e-5, per_device_train_batch_size=16,
    per_device_eval_batch_size=16, num_train_epochs=1,
    weight_decay=0.01,
    report_to="none",
)

In [11]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    # Summary metrics
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)

    # Confusion matrix
    cm = confusion_matrix(labels, preds)
    print("\nConfusion Matrix:")
    print(cm)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
    }

In [12]:
#Step 5: Fine-Tune the Model
from transformers import Trainer
# Create Trainer instance
trainer = Trainer (
model=model, args=training_args, train_dataset=tokenized_datasets["train"],
eval_dataset=tokenized_datasets["test"],compute_metrics=compute_metrics)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Confusion Matrix:
[[11515   985]
 [  892 11608]]


TrainOutput(global_step=782, training_loss=0.10161624235265396, metrics={'train_runtime': 952.4988, 'train_samples_per_second': 26.247, 'train_steps_per_second': 0.821, 'total_flos': 3311684966400000.0, 'train_loss': 0.10161624235265396, 'epoch': 1.0})

In [13]:
#Step 6: Evaluate the Model
# Evaluate the fine-tuned model
trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Confusion Matrix:
[[11515   985]
 [  892 11608]]


{'eval_loss': 0.25812581181526184,
 'eval_accuracy': 0.92492,
 'eval_f1': 0.9251982624636352,
 'eval_precision': 0.921781942348924,
 'eval_recall': 0.92864,
 'eval_runtime': 240.2183,
 'eval_samples_per_second': 104.072,
 'eval_steps_per_second': 3.255,
 'epoch': 1.0}

## Inference

In [14]:
import torch
import torch.nn.functional as F

def predict_sentiment(text):
    model.eval()  # Set model to eval mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Make sure model is on the correct device

    # Tokenize and move input to the same device
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move input tensors to device

    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()
        confidence = probs[0][predicted_class].item()

    sentiment = "Positive 😊" if predicted_class == 1 else "Negative 😠"
    print(f"\nText: {text}")
    print(f"Predicted Sentiment: {sentiment} (Confidence: {confidence:.2f})")


In [15]:
predict_sentiment("I absolutely loved the movie, it was fantastic!")
predict_sentiment("This was the worst film I’ve ever seen.")


Text: I absolutely loved the movie, it was fantastic!
Predicted Sentiment: Positive 😊 (Confidence: 0.99)

Text: This was the worst film I’ve ever seen.
Predicted Sentiment: Negative 😠 (Confidence: 1.00)


#### After training, you can save your model for later inference:

model.save_pretrained("./sentiment-model")

tokenizer.save_pretrained("./sentiment-model")



#### To load model in future

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained("./sentiment-model")

tokenizer = DistilBertTokenizer.from_pretrained("./sentiment-model")
